https://imgur.com/gQuqXDC

In [ ]:
# Import dependencies
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.figure_factory as ff
from plotly import subplots
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from datetime import date
from fbprophet import Prophet
import math


In [ ]:
#conf_rw = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
#covid.columns = ['STATE','COUNTRY','LAT','LONG','DATE','CONFIRMED','DEATH', 'RECOVERED']
#covid['DATE'] = covid['DATE'].apply(pd.to_datetime).dt.normalize()
#covid.head()

In [ ]:
# Importing data
covid = pd.read_csv("data\\covid_19_data.csv", parse_dates=['ObservationDate'])
conf_rw = pd.read_csv("data\\time_series_covid_19_confirmed.csv")

# Formating covid dataset reported from WHO
covid = covid.drop('SNo', axis =1)
covid.columns = ['DATE', 'STATE','COUNTRY','LAST UPDATED','CONFIRMED','DEATH', 'RECOVERED']
covid['DATE'] = covid['DATE'].apply(pd.to_datetime).dt.normalize() 
covid['LAST UPDATED'] = covid['LAST UPDATED'].apply(pd.to_datetime).dt.normalize()

# Selecting subset of information needed in conf_rw dataframe
conf_rw = conf_rw.loc[:,['Province/State','Country/Region','Lat','Long']]
conf_rw.columns = ['STATE','COUNTRY','LAT','LONG']

# Creating covid_geo data frame that includes long and lat. 
covid = pd.merge(covid,conf_rw[['STATE','LAT','LONG']],on='STATE', how='left')

# Changing Mainland China into China
covid['COUNTRY'].replace({'Mainland China':'China'},inplace=True)

covid.head()

In [ ]:
# Produce quick summary with total numbers
summary_all = covid.groupby('DATE')['CONFIRMED', 'DEATH', 'RECOVERED'].sum().reset_index()
summary = summary_all.sort_values('DATE', ascending=False)

ct_sum = covid['COUNTRY'].unique().tolist()
print("As of now there is" +" " + str(len(ct_sum)) +" " + "countries to which virus has spread.")

summary.head(1).style.background_gradient(cmap='OrRd')

### Checking Data Integrity with pandas_profiling library
+ Pandas Profiling provides quick way to understand data set we are workign with.

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(covid)

In [ ]:
mis_coord = pd.isnull(covid["LAT"]) 
temp = covid[mis_coord]
temp

In [ ]:
temp.COUNTRY.unique()

In [ ]:
# Spread, death and recovered over the time in Serbia

sr_sum = sr_sum[sr_sum['DATE'] > '2020-03-01']

fig = make_subplots(rows=1, cols=3, subplot_titles=(f"{int(sr_sum.CONFIRMED.max()):,d}" +' ' + "CONFIRMED",
                                                    f"{int(sr_sum.DEATH.max()):,d}" +' ' + "DEATHS",
                                                    f"{int(sr_sum.RECOVERED.max()):,d}" +' ' + "RECOVERED"))

trace1 = go.Scatter(
                x=sr_sum['DATE'],
                y=sr_sum['CONFIRMED'],
                name="CONFIRMED",
                line_color='orange',
                opacity=0.8)
trace2 = go.Scatter(
                x=sr_sum['DATE'],
                y=sr_sum['DEATH'],
                name="DEATH",
                line_color='dimgray',
                opacity=0.8)

trace3 = go.Scatter(
                x=sr_sum['DATE'],
                y=sr_sum['RECOVERED'],
                name="RECOVERED",
                line_color='deepskyblue',
                opacity=0.8)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.update_layout(template="ggplot2",title_text = '<b>Spread Vs. Death Vs Recovered within Serbia </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'), showlegend=False)
fig.show()

In [ ]:
sr_sum = sr_sum.sort_values('DATE', ascending=True)
sr_sum = pd.DataFrame(sr_sum.set_index('DATE').diff()).reset_index()
sr_sum.dropna()
sr_sum.head()

In [ ]:
sr_sum.describe()

In [ ]:
# Ploting daily updtes for 
fig_d = go.Figure()
fig_d.add_trace(go.Scatter(x=sr_sum.DATE, y=sr_sum.CONFIRMED, mode="lines+markers", name=f"MAX. OF {int(sr_sum.CONFIRMED.max()):,d}" + ' ' + "CONFIRMED",line_color='Orange'))
fig_d.add_trace(go.Scatter(x=sr_sum.DATE, y=sr_sum.RECOVERED, mode="lines+markers", name=f"MAX. OF {int(sr_sum.RECOVERED.max()):,d}" + ' ' + "RECOVERED",line_color='deepskyblue'))
fig_d.add_trace(go.Scatter(x=sr_sum.DATE, y=sr_sum.DEATH, mode="lines+markers", name=f"MAX. OF {int(sr_sum.DEATH.max()):,d}" + ' ' + "DEATHS",line_color='dimgray'))
fig_d.update_layout(template="ggplot2",title_text = '<b>Daily numbers for Confirmed, Death and Recovered </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'), showlegend=True)
fig_d.update_layout(
    legend=dict(
        x=0.01,
        y=.98,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="Black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="dimgray",
        borderwidth=2
    ))
fig_d.show()

In [ ]:
D_vs_R = sr_sum.copy()
D_vs_R['REC'] = 'REC'
D_vs_R['DTH'] = 'DTH'
recovered = pd.pivot_table(D_vs_R.dropna(subset=['RECOVERED']), index='DATE', 
                         columns='REC', values='RECOVERED', aggfunc=np.sum).fillna(method='ffill').reset_index()

death = pd.pivot_table(D_vs_R.dropna(subset=['DEATH']), index='DATE', 
                         columns='DTH', values='DEATH', aggfunc=np.sum).fillna(method='ffill').reset_index()

D_vs_R_df = pd.merge(recovered,death,on='DATE')
D_vs_R_df['RATIO'] = round(D_vs_R_df['DTH'] / (D_vs_R_df['DTH'] + D_vs_R_df['REC'])*100)
D_vs_R_df.tail()

In [ ]:
# Display number of confirmed cased in Serbia
sr_max = covid[(covid['COUNTRY'] == 'Serbia')]
fig_conf = go.Figure()
fig_conf.add_trace(go.Scatter(x=sr_max.DATE,
                              y=sr_max.CONFIRMED,
                              mode="lines+markers",
                              line_color='Orange',
                              name = f"{int(sr_max.CONFIRMED.max()):,d}" + ' ' + 'Confirmed'))

fig_conf.update_layout(template="ggplot2",title_text = '<b>Confiermed cases in Serbia</b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans", color='black'), showlegend=True) 
fig_conf.update_layout(
    legend=dict(
        x=0.01,
        y=.98,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="Black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Orange",
        borderwidth=2
    ))
fig_conf.show()

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.plot import plot_plotly


sr_max = sr_max.groupby(['DATE']).sum().loc[:,['CONFIRMED','DEATH','RECOVERED']].reset_index()
sr_max = sr_max[sr_max['DATE'] > '2020-03-1']

sr_prophet= sr_max.rename(columns={'DATE': 'ds', 'CONFIRMED': 'y'})
# Make a future dataframe for X days
m_sr = Prophet(
    changepoint_prior_scale=20,
    seasonality_prior_scale=20,
    n_changepoints=19,
    changepoint_range=0.9,
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = True,
    seasonality_mode = 'additive')
# Add seasonlity
#m_us.add_seasonality(name='yearly', period=365, fourier_order=5)
m_sr.fit(sr_prophet)

# Make predictions
future_sr = m_sr.make_future_dataframe(periods=5)

forecast_sr = m_sr.predict(future_sr)

In [ ]:
trace1 = {
  "fill": None, 
  "mode": "markers", 
  "name": "actual no. of Confirmed", 
  "type": "scatter", 
  "x": sr_prophet.ds, 
  "y": sr_prophet.y
}
trace2 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "upper_band", 
  "type": "scatter", 
  "x": forecast_sr.ds, 
  "y": forecast_sr.yhat_upper
}
trace3 = {
  "fill": "tonexty", 
  "line": {"color": "#57b8ff"}, 
  "mode": "lines", 
  "name": "lower_band", 
  "type": "scatter", 
  "x": forecast_sr.ds, 
  "y": forecast_sr.yhat_lower
}
trace4 = {
  "line": {"color": "#eb0e0e"}, 
  "mode": "lines+markers", 
  "name": "prediction", 
  "type": "scatter", 
  "x": forecast_sr.ds, 
  "y": forecast_sr.yhat
}

data = [trace1, trace2, trace3, trace4]
layout = {
  "title": "Confirmed - Time Series Forecast - Daily Trend", 
  "xaxis": {
    "title": "", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
  "yaxis": {
    "title": "Confirmed nCov - US", 
    "ticklen": 5, 
    "gridcolor": "rgb(255, 255, 255)", 
    "gridwidth": 2, 
    "zerolinewidth": 1
  }, 
}
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
# Produce quick summary for Serbia with total numbers
sr_df =  covid[(covid['COUNTRY'] == 'Serbia')]
sr_sum = sr_df.groupby('DATE')['CONFIRMED', 'DEATH', 'RECOVERED'].sum().reset_index()
sr_sum = sr_sum.sort_values('DATE', ascending=False)
sr_sum.head(1).style.background_gradient(cmap='OrRd')